In [ ]:
# Create a new Django project by running the following command in a Jupyter Notebook cell:
!django-admin startproject myproject

In [ ]:
# Create a new Django app by running the following command:
!python manage.py startapp shop

In [ ]:
# Open the myproject/settings.py file and add 'shop' to the INSTALLED_APPS list:
# myproject/settings.py
INSTALLED_APPS = [
    # ...
    'shop',
]


In [ ]:
# In the shop/models.py file, define the models for your shop:
# shop/models.py
from django.db import models

class Product(models.Model):
    name = models.CharField(max_length=200)
    description = models.TextField()
    price = models.DecimalField(max_digits=10, decimal_places=2)

class Order(models.Model):
    created = models.DateTimeField(auto_now_add=True)
    customer_name = models.CharField(max_length=200)
    customer_email = models.EmailField()
    products = models.ManyToManyField(Product, through='OrderItem')

class OrderItem(models.Model):
    order = models.ForeignKey(Order, on_delete=models.CASCADE)
    product = models.ForeignKey(Product, on_delete=models.CASCADE)
    quantity = models.PositiveIntegerField()


In [ ]:
# Create the database tables by running the following commands in separate cells:
!python manage.py makemigrations
!python manage.py migrate


In [ ]:
# In the shop/views.py file, create views for displaying the products and handling orders:
# shop/views.py
from django.shortcuts import render, get_object_or_404
from django.views.generic.list import ListView
from django.views.generic.edit import FormView
from .models import Product, Order, OrderItem

class ProductListView(ListView):
    model = Product
    template_name = 'product_list.html'

class OrderFormView(FormView):
    template_name = 'order_form.html'
    form_class = OrderForm
    success_url = '/thankyou/'

    def form_valid(self, form):
        order = Order.objects.create(
            customer_name=form.cleaned_data['name'],
            customer_email=form.cleaned_data['email']
        )
        for product_id, quantity in form.cleaned_data['products'].items():
            product = get_object_or_404(Product, id=product_id)
            OrderItem.objects.create(
                order=order,
                product=product,
                quantity=quantity
            )
        return super().form_valid(form)


In [ ]:
# Create the templates for the views in shop/templates/:
# product_list.html

{% extends 'base.html' %}

{% block content %}
    <h1>Products</h1>
    <ul>
    {% for product in object_list %}
        <li>
            <a href="{% url 'product_detail' product.pk %}">{{ product.name }}</a>
            - ${{ product.price }}
        </li>
    {% empty %}
        <li>No products available</li>
    {% endfor %}
    </ul>
{% endblock %}


In [ ]:
# `order_form.html`
{% extends 'base.html' %}

{% block content %}
    <h1>Order form</h1>
    <form method="post">
        {% csrf_token %}
        {{ form.as_p }}
        <input type="submit" value="Place order">
    </form>
{% endblock %}


In [ ]:
# 8.Define the URLs for the views in shop/urls.py:
# shop/urls.py
from django.urls import path
from .views import ProductListView, OrderFormView